In [59]:
import pyomo.environ as pyo
solver = 'appsi_highs'
SOLVER = pyo.SolverFactory(solver)
assert SOLVER.available(), f"Solver {solver} is not available."

In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

#### Load Data

In [84]:
df = pd.read_csv("duopoly_competition_details.csv")
df = df[(df['selling_season'] == df['selling_season'][0]) & (df['competition_id'] == df['competition_id'][0])]
X = df[[ 'price', 'price_competitor']]
X['intercept'] = 1
y = df[['demand']] 

X = X.to_numpy()
y = np.asarray(y).reshape(-1)
# plt.plot(X[0:], y)
print(X, y)

[[33.5 55.5  1. ]
 [31.8 33.5  1. ]
 [30.2 31.8  1. ]
 [28.7 30.2  1. ]
 [37.7 28.7  1. ]
 [35.9 37.7  1. ]
 [34.1 35.9  1. ]
 [32.4 34.1  1. ]
 [30.7 32.4  1. ]
 [33.7 30.7  1. ]
 [32.1 33.7  1. ]
 [30.5 32.1  1. ]
 [28.9 30.5  1. ]
 [27.5 28.9  1. ]
 [31.2 27.5  1. ]
 [29.6 31.2  1. ]
 [28.1 29.6  1. ]
 [26.7 28.1  1. ]
 [25.4 26.7  1. ]
 [28.6 25.4  1. ]
 [27.2 28.6  1. ]
 [25.8 27.2  1. ]
 [24.6 25.8  1. ]
 [23.3 24.6  1. ]
 [26.3 23.3  1. ]
 [25.  26.3  1. ]
 [23.8 25.   1. ]
 [22.6 23.8  1. ]
 [21.4 22.6  1. ]
 [24.2 21.4  1. ]
 [23.  24.2  1. ]
 [21.8 23.   1. ]
 [20.7 21.8  1. ]
 [20.  20.7  1. ]
 [22.2 20.   1. ]
 [21.1 22.2  1. ]
 [20.1 21.1  1. ]
 [20.  20.1  1. ]
 [20.  20.   1. ]
 [20.7 20.   1. ]
 [20.  20.7  1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.1 20.   1. ]
 [20.  20.1  1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.   1. ]
 [20.  20.

C:\Users\yannl\AppData\Local\Temp\ipykernel_17480\3348549926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1


## LAD

In [85]:
def lad_regression(X, y):
    model = pyo.ConcreteModel("LAD Regression")

    # get dimensions of data
    n, k = X.shape

    # create index sets (note use of Python style zero based indexing)
    model.I = pyo.RangeSet(0, n - 1)
    model.J = pyo.RangeSet(0, k - 1)

    # create variables
    model.ep = pyo.Var(model.I, domain=pyo.NonNegativeReals)
    model.em = pyo.Var(model.I, domain=pyo.NonNegativeReals)
    model.m = pyo.Var(model.J)
    model.b = pyo.Var()

    # constraints
    @model.Constraint(model.I)
    def residuals(m, i):
        return m.ep[i] - m.em[i] == y[i] - sum(X[i][j] * m.m[j] for j in m.J) - m.b

    # objective
    @model.Objective(sense=pyo.minimize)
    def sum_of_abs_errors(m):
        return sum(m.ep[i] + m.em[i] for i in m.I)

    SOLVER.solve(model)

    return model


model = lad_regression(X, y)
print(
    f"Optimal solution: m = {[f'{model.m[j].value:.2f}' for j in model.J]} and b = {model.b.value:.2f}"
)
print(f"Objective value: {pyo.value(model.sum_of_abs_errors):.2f}")

Optimal solution: m = ['-0.00', '-0.00', '-211040465.72'] and b = 211040465.72
Objective value: 37.00


## OLS

In [63]:
solver = 'ipopt'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)
assert SOLVER.available(), f"Solver {solver} is not available."

In [64]:
def ols_regression(X, y):
    m = pyo.ConcreteModel("OLS Regression")

    n, k = X.shape

    # note use of Python style zero based indexing
    m.I = pyo.RangeSet(0, n - 1)
    m.J = pyo.RangeSet(0, k - 1)

    m.e = pyo.Var(m.I, domain=pyo.Reals)
    m.m = pyo.Var(m.J)
    m.b = pyo.Var()

    @m.Constraint(m.I)
    def residuals(m, i):
        return m.e[i] == y[i] - sum(X[i][j] * m.m[j] for j in m.J) - m.b

    @m.Objective(sense=pyo.minimize)
    def sum_of_square_errors(m):
        return sum((m.e[i]) ** 2 for i in m.I)

    return m


m = ols_regression(X, y)
SOLVER.solve(m)
m.m.display()
m.b.display()

m : Size=2, Index=J
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
      0 :  None : -0.021120745728782243 :  None : False : False :  Reals
      1 :  None : 0.0027597065081516443 :  None : False : False :  Reals
b : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 0.7860362269685683 :  None : False : False :  Reals
